In [ ]:
import tensorflow        as tf
import numpy             as np

import pathlib
import random
import data
import os

from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing             import image
from tensorflow.keras.models                    import load_model

In [ ]:
dataObj    = data.Data()
numClasses = dataObj.numClasses

In [ ]:
def getBestSavedModel(modelCheckpointPath):
    dirPaths  = modelCheckpointPath.glob("*")
    minLoss   = float('inf')
    bestModel = ""
    for dirPath in dirPaths:
        modelname = pathlib.Path(dirPath.name).stem
        loss = float(modelname.split("_")[-1])
        if loss <= minLoss:
            bestModel = dirPath
            minLoss   = loss
    return str(bestModel)

In [ ]:
def predictCNN():
    def getPredictions(model, framePath):
        frame = image.load_img(framePath, 
                               target_size   = (299,299), 
                               interpolation = "lanczos")   # shape: (299, 299, 3): # of dim: 3
        frame_arr   = image.img_to_array(frame)             # pixel values in range [0,255]
        frame_arr   = preprocess_input(frame_arr)           # pixel values in range [-1, 1]
        frame_arr   = np.expand_dims(frame_arr, axis = 0)   # expands shape to: (1, 299, 299, 3): # of dim: 4
        predictions = model.predict(frame_arr)
        predictions = predictions[0]
        return predictions
    
    
    dataSetType   = "Test"
    rootPath      = pathlib.Path(r"D:\ActionRecognition")
    dataDirectory = rootPath/'Frames'/dataSetType
    framesPaths   = list(dataDirectory.glob('*/*.jpg'))
    frameCount    = len(framesPaths)
    
    modelCheckpointPath = rootPath/'Callbacks\CNN'/f'{numClasses}'/'ModelCheckpoint'
    savedModelPath      = "" # you can insert the path to a saved model here
    if savedModelPath == "":
        savedModelPath = getBestSavedModel(modelCheckpointPath)
    savedModel = load_model(savedModelPath)
    
    
    numFrames = 5
    for i in range(numFrames):
        randomIndex     = random.randint(0, frameCount - 1)
        randomFramePath = framesPaths[randomIndex]
        predictions     = getPredictions(savedModel, randomFramePath)
        print(predictions)
        
predictCNN()

In [ ]:
def predictRNN():
    def getPredictions(model, sequencePath):
        maxFrameCount = dataObj.getMaxFrameCount()
        sequence      = np.load(sequencePath)
        sequence      = np.pad(sequence, ((0, maxFrameCount - len(sequence)), (0, 0)), 'edge')
        sequence      = np.expand_dims(sequence, axis = 0)
        predictions   = model.predict(sequence)
        predictions   = predictions[0]
        return predictions
    
    dataSetType    = "Test"
    rootPath       = pathlib.Path(r"D:\ActionRecognition")
    dataDirectory  = rootPath/'Sequences'/dataSetType
    sequencesPaths = list(dataDirectory.glob('*/*.npy'))
    sequenceCount  = len(sequencesPaths)
    
    modelCheckpointPath = rootPath/'Callbacks\RNN'/f'{numClasses}'/'ModelCheckpoint'
    savedModelPath      = "" # you can insert the path to a saved model here
    if savedModelPath == "":
        savedModelPath = getBestSavedModel(modelCheckpointPath)
    savedModel = load_model(savedModelPath)
    
    numSequences = 5
    for i in range(numSequences):
        randomIndex        = random.randint(0, sequenceCount - 1)
        randomSequencePath = sequencesPaths[randomIndex]
        pathStem           = randomSequencePath.stem
        partsOfPathStem    = pathStem.split("_")
        print("Video: ", "_".join(randomSequencePath.stem.split("_")[:-1]))
        print("Actual class: ", partsOfPathStem[1])
        predictions        = getPredictions(savedModel, randomSequencePath)
        print(predictions)
        print()
        
predictRNN()